In [1]:
import sys
from os import getcwd
from os.path import join
import pandas as pd
import numpy as np
from IPython.display import display
import seaborn as sns

sys.path.insert(0, join(getcwd(), "../module_code"))

from data.load import load_data, load_outcomes
from data.utils import read_files_and_combine
from cli_utils import load_cli_args, init_cli_args

sys.argv = [sys.argv[0]]
load_cli_args("../options.yml")
args = init_cli_args()

/home/davina/mambaforge/envs/crrt/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Vitals

In [ ]:
vitals = "Flowsheet_Vitals.txt"
crrt_df = read_files_and_combine([vitals], args.ucla_crrt_data_dir)
ctrl_df = read_files_and_combine([vitals], args.ucla_control_data_dir)
cedars_df = read_files_and_combine([vitals], args.cedars_crrt_data_dir)

In [ ]:
crrt_df[crrt_df["VITAL_SIGN_TYPE"].str.lower().str.contains("weight")]

In [ ]:
cedars_df[cedars_df["MEAS_NAME"].str.lower().str.contains("weight")]

# Labs

In [2]:
from data.longitudinal_features import map_labs

labs = "Labs.txt"

crrt_df = read_files_and_combine([labs], args.ucla_crrt_data_dir)
ctrl_df = read_files_and_combine([labs], args.ucla_control_data_dir)
cedars_df = read_files_and_combine([labs], args.cedars_crrt_data_dir).rename(
    {"RESULT": "RESULTS", "NAME": "COMPONENT_NAME"}, axis=1
)
cedars_df = map_labs(cedars_df, args.cedars_crrt_data_dir)

mealy | WARNING - Unexpected encoding in Labs.txt. Encoding with cp1252.


In [3]:
# ucla_units = crrt_df.groupby("DESCRIPTION")["REFERENCE_UNIT"]
ucla_units = crrt_df.groupby("COMPONENT_NAME")["REFERENCE_UNIT"]
cedars_units = cedars_df.groupby("COMPONENT_NAME")["REFERENCE_UNIT"]

In [4]:
def del_key(key, d: dict) -> dict:
    if d:
        d.pop(key, None)
    return d


def units_per_lab(group: pd.Series) -> bool:
    # a = group.dropna().str.lower()
    a = group.dropna().str.lower().value_counts()
    if not group.isna().all():
        # mode = a.mode().values[0]
        mode = a.idxmax()
        mode_count = a[mode]
        if (a.iloc[0] == a).all():
            return {"mode_unit": {mode: mode_count}, "other_ucla_units": {}}
        else:
            return {
                "mode_unit": {mode: mode_count},
                "other_ucla_units": del_key(mode, a.to_dict()),
            }
    return None


ucla_unmatched_internal = pd.DataFrame(ucla_units.agg(units_per_lab).to_dict()).T
ucla_unmatched_internal

,mode_unit,other_ucla_units
"O2 SATURATION/MEASURED,ECMO PREMEMBRANE",{'%': 1178},{}
% CD16+CD56 (NK CELLS),{'%': 2},{}
% CD19 (B CELLS),{'%': 2},{}
% CD19 (B CELLS)(QST),{'%': 1},{}
% CD3 (MATURE T CELLS)(QST),{'%': 3},{}
...,...,...
"ZINC,SERUM",{'mcg/ml': 842},{}
"ZINC,SERUM(OSL)",None,None
"ZZBENZODIAZEPINES,SCREEN,SERUM",None,None
"ZZTRICYCLIC AD,SCREEN,SERUM",None,None


In [ ]:
# cedar_units_sets = cedars_units.agg(
#     lambda group: set(group.dropna().str.lower()) if not group.isna().all() else set()
# ).rename("cedars units")
cedars_unit_counts = cedars_units.agg(
    lambda group: group.dropna().str.lower().value_counts().to_dict()
    if not group.isna().all()
    else {}
).rename("cedars_units")
cedars_unit_counts

In [ ]:
combined = (
    ucla_unmatched_internal.dropna(subset="other_ucla_units")
    .join(cedars_unit_counts)
    .replace({"cedars_units": {np.nan: {}}})
)


def add_mode_count_from_cedars(row):
    mode = next(iter(row["mode_unit"]))
    count = row["mode_unit"][mode]
    count += row["cedars_units"].get(mode, 0)
    return {mode: count}


combined["mode_unit"] = combined.apply(add_mode_count_from_cedars, axis=1)
combined["cedars_units"] = combined.apply(
    lambda row: del_key(next(iter(row["mode_unit"])), row["cedars_units"]), axis=1
)
combined["combined_units"] = pd.Series(
    [
        # set1.union(set2) - {mode}
        # sum the counts
        # https://stackoverflow.com/a/10461916/1888794
        del_key(
            next(iter(mode)),
            {k: set1.get(k, 0) + set2.get(k, 0) for k in set(set1) | set(set2)},
        )
        for set1, set2, mode in zip(
            combined["other_ucla_units"],
            combined["cedars_units"],
            combined["mode_unit"],
        )
    ],
    index=combined.index,
)
combined

In [ ]:
# Columns that are actually used by the model with different units
import pickle

columns_path = join("..", "local_data", "columns.pkl")
with open(columns_path, "rb") as f:
    selected_cols = pickle.load(f)
colnames = selected_cols.str.rstrip("_len|max|min|skew|std").unique()
selected_cols_with_units = combined.loc[colnames.intersection(combined.index)]
selected_cols_with_units[selected_cols_with_units["combined_units"] != {}]

In [ ]:
# any_mismatch_units = selected_cols_with_units[selected_cols_with_units["combined_units"] != {}]
# any_mismatch_units
any_mismatch_units = combined[combined["combined_units"] != {}]
any_mismatch_units

Unit conversion resources:
- [LOINC Table CSV](https://github.com/shihjay2/nosh2/blob/master/resources/LOINC.csv)
- [LOINC Web Lookup](https://www.vas.ehealth.fgov.be/webretam/retam/home.htm?eventName=LOINC_FILTER_APPLY&htmlfield_analysesId=&htmlfield_assoc=no&htmlfield_filter1=5403-1&htmlfield_searchin_filter1=all&htmlfield_filter2=&htmlfield_searchin_filter2=all&htmlfield_filter3=&htmlfield_searchin_filter3=all)
- [Units and Abbreviations](https://www.aruplab.com/files/resources/testing/KeyUnitsAbbrev.pdf)
- [Unit string variations](https://www.cdc.gov/cliac/docs/addenda/cliac0313/13A_CLIAC_2013March_UnitsOfMeasure.pdf)

In [ ]:
equivalents = [  # map mode to others
    {"/hpf", "per hpf"},
    {"ml", "ml/24 h"},  # 24 HR. URINE VOLUME
    {"cells/ul", "/ul"},  # WBCS and others
    {"%", "% normal"},  # VWF: RISTOCETIN CO-FACTOR
    {"%", "% binding inhibition"},
    {"%", "% baseline"},
    {"%", "% of total"},
    {"%", "% activity"},
    {"%", "g/dl"},
    {"cpy/ml", "copiesml"},
    {"uge/ml", "mcg eq/ml", "ug eq/ml"},
    {"units/ml", "% normal"},
    {"mg/dl", "mg/dl adult"},
    {"au/ml", "u/ml", "titer"},
    {"ng/ml feu", "ng/mlfeu"},
    {"ml/min/1.73m2", "ml/min/bsa", "ml/min/1.73", "ml/min/1.73m", "ml/min"},
    {"mg/dl", "md/dl", "mg/di"},  # typo
    {"ng/ml", "ng/ml rbc"},
    {"au/ml", "ai"},
    {"u/g hgb", "u/g hb"},
    {"liv", "index"},  # LYME DISEASE AB,TOTAL
    {"ng/ml", "eia unit"},
    # ("fl = femtoliter = cu mic = cubic micron")
    {"fl", "cu mic"},
    {"nmol bce/mmol creat", "nm bce/mm creat"},
    {"m2", "meters s"},
    {"pg/mlcdf", "pg/ml"},
    {"m/ul", "mi/mm3", "m/mm3", "x10e6/ul"},
    {"iv", "IV", "index", "INDEX"},
]
# https://www.cdc.gov/cliac/docs/addenda/cliac0313/13A_CLIAC_2013March_UnitsOfMeasure.pdf
count_units = {
    "x10e3": 1e3,
    "thousand": 1e3,
    "thous": 1e3,
    "1000": 1e3,
    "10e3": 1e3,
    "k": 1e3,
    "x10e6": 1e6,
    "10e6": 1e6,
    "mill": 1e6,
    "million": 1e6,
    "X10*9": 1e9,
    "x10*9": 1e9,
    "x10*12": 1e12,
    "cells": 1,
}
mol_to_eq = {  # meq/mol
    "ANION GAP": 1,
    "BASE EXCESS": 1,
    "BICARBONATE": 1,
    "CARBON DIOXIDE": 1,
    "CHLORIDE": 1,
    "MAGNESIUM": 2,
    "PHOSPHORUS": 3,  # 3 or 5
    "POTASSIUM": 1,
    "SODIUM": 1,
}
g_to_mol = {  # inverse of molecular weight, so mol/g
    "AMMONIA": 0.058719906,
    "BETA-HYDROXYBUTYRATE": 0.009605,
    "CREATININE": 0.008842,
    "CREATININE, RANDOM URINE": 0.008842,
    "CREATININE,RANDOM URINE": 0.008842,
    "LIPOPROTEIN (A)": 0.0028011204,
    "MAGNESIUM": 0.0411438,
    "METANEPHRINE": 0.0050702226,
    "METANEPHRINE, FREE PLASMA": 0.0050702226,
    "NORMETANEPHRINE, FREE PLASMA": 0.0050702226,
    "PTH-RELATED PROTEIN (PTH-RP)": 0.0001061008,
    "VITAMIN B6": 0.0059101655,
    "VITAMIN C": 0.0056818182,
}
# timed + LIPOPROTEIN (A)

In [ ]:
from pint import UnitRegistry, Quantity, UndefinedUnitError, DimensionalityError

ureg = UnitRegistry()
# https://pint.readthedocs.io/en/stable/advanced/defining.html#programmatically
ureg.define("micro- = 1e-6 = u = mc")
ureg.define("iu = u")
ureg.define("mm3 = mm**3")
ureg.define("eq = equivalent")


def convert(a, b) -> Quantity:
    return ureg(a).to(b)


convert("10 iu/l", "u/l")

In [ ]:
convert("10 iu/l", "u/l").magnitude

In [ ]:
def cant_auto_convert(row):
    mode = next(iter(row["mode_unit"]))
    cant_convert = {}
    for other_unit, count in row["combined_units"].items():
        try:
            if not any(
                mode in equiv_set and other_unit in equiv_set
                for equiv_set in equivalents
            ):
                # time
                def proc_unit(unit):
                    # drop (calc) and creat
                    unit = unit.replace("(calc)", "")
                    unit = unit.replace("creat", "")
                    unit = unit.replace("crt", "")

                    if "/24" in unit:
                        return unit.split("/")[0] + "/d"
                    return unit

                mode = proc_unit(mode)
                other_unit = proc_unit(other_unit)

                # import difflib
                conv_eq_to_mol = ("mol" in mode or "g" in mode) and "eq" in other_unit
                conv_mol_to_eq = "eq" in mode and "mol" in other_unit
                conv_g_to_eq = "eq" in mode and "g" in other_unit

                # do i need valence/charge
                if conv_eq_to_mol or conv_mol_to_eq:
                    print("[[MOL <-> EQ]]")
                    name = next(iter([k for k in mol_to_eq if k in row.name]), None)
                    if conv_eq_to_mol:
                        print(
                            f"Convert {other_unit} to {mode} for {row.name}: val * {1/mol_to_eq[name]}"
                        )
                        other_unit = other_unit.replace("eq", "mol")
                    elif conv_mol_to_eq:
                        print(
                            f"Convert {other_unit} to {mode} for {row.name}: val * {mol_to_eq[name]}"
                        )
                        other_unit = other_unit.replace("mol", "eq")
                # print("\n".join(difflib.ndiff([str(mode)], ["x10e3/ul"])))
                conv_g_to_mol = "mol" in mode and "g" in other_unit
                conv_mol_to_g = "g" in mode and "mol" in other_unit
                # pick longest match
                count_unit_convert_mode = sorted(
                    [count for count in count_units if count in mode],
                    key=len,
                    reverse=True,
                )
                count_unit_convert_other = sorted(
                    [count for count in count_units if count in other_unit],
                    key=len,
                    reverse=True,
                )

                # do i need molecular weight
                if (conv_g_to_mol or conv_g_to_eq) or conv_mol_to_g:
                    print("[[MOL <-> G]]")
                    if conv_g_to_eq:
                        print("[[MOL <-> EQ]]")
                        name = next(iter([k for k in mol_to_eq if k in row.name]), None)
                        v = convert(
                            other_unit.replace("g", "mol"), mode.replace("eq", "mol")
                        )
                        print(
                            f"Convert {other_unit} to {mode} for {row.name}: val * {v} * {g_to_mol[row.name]} mol/g * {mol_to_eq[name]} eq/mol"
                        )
                    elif conv_g_to_mol:
                        v = convert(other_unit.replace("g", "mol"), mode)
                        print(
                            f"Convert {other_unit} to {mode} for {row.name}: val * {v} * {g_to_mol[row.name]} mol/g"
                        )
                    elif conv_mol_to_g:
                        v = convert(other_unit.replace("mol", "g"), mode)
                        name = next(iter([k for k in g_to_mol if k in row.name]), None)
                        print(
                            f"Convert {other_unit} to {mode} for {row.name}: val * {v} * {1/g_to_mol.get(name, -1)} g/mol"
                        )
                elif count_unit_convert_mode or count_unit_convert_other:
                    print("[[Named Units]]")
                    to_convert_mode = next(iter(count_unit_convert_mode), "")
                    to_convert_other = next(iter(count_unit_convert_other), "")
                    other_name = (
                        other_unit.replace(to_convert_other, "u")
                        if to_convert_other
                        else other_unit
                    )
                    mode_name = (
                        mode.replace(to_convert_mode, "u") if to_convert_mode else mode
                    )
                    v = convert(other_name, mode_name)
                    print(
                        f"Convert {other_unit} to {mode} for {row.name}: val * {v} * {count_units.get(to_convert_other,1)} / {count_units.get(to_convert_mode, 1)}"
                    )
                else:
                    convert(other_unit, mode)
            else:
                print("[[EQUIV]]")
                print(
                    f"{other_unit} and {mode} for {row.name} are considered equivalent."
                )
        except UndefinedUnitError as e:
            print(f"[[ERROR]] ({row.name})\n{e}")
            cant_convert[other_unit] = count
        except DimensionalityError as e:
            print(f"[[ERROR]] ({row.name})\n{e}")
            cant_convert[other_unit] = count
        except ValueError as e:
            print(f"[[ERROR]] ({row.name})\n{e}")
            cant_convert[other_unit] = count
        except:
            print(row)
            cant_convert[other_unit] = count
    return cant_convert

In [ ]:
# combined = combined.assign(cant_convert=lambda df: df.apply(cant_auto_convert, axis=1))
selected_cols_with_units = selected_cols_with_units.assign(
    cant_convert=lambda df: df.apply(cant_auto_convert, axis=1)
)

In [ ]:
# cant_convert = combined[combined["cant_convert"] != {}]
cant_convert = selected_cols_with_units[selected_cols_with_units["cant_convert"] != {}]
cant_convert.sort_values(
    by="mode_unit",
    ascending=False,
    key=lambda x: x.map(lambda y: next(iter(y.values()))),
)

In [ ]:
# selected_cols_with_units.to_csv("lab_units_selected_columns.csv")

## Manually Check Equivalents

In [ ]:
def plot_range(df: pd.DataFrame, match_str: str):
    data = df[
        df["COMPONENT_NAME"].isin(
            any_mismatch_units.index[any_mismatch_units.index.str.match(match_str)]
        )
    ]
    try:
        data["RESULTS"] = pd.to_numeric(data["RESULTS"])
    except:
        coerced = pd.to_numeric(data["RESULTS"], errors="coerce")
        display(data[coerced.isna()].groupby("RESULTS").all())
        display(data.groupby("REFERENCE_UNIT")["RESULTS"].describe())
        data["RESULTS"] = coerced  # .fillna(-100)

    sns.catplot(
        x="REFERENCE_UNIT",
        y="RESULTS",
        data=data,
    )
    return data


# x = plot_range(crrt_df, "ABSOLUTE.*(QST)")
# The ranges are not the same. "X10*9/L" is between 0 and 10, the other is between 0 and 3x10e5.
# tmp = x[x["REFERENCE_UNIT"] == "X10*9/L"][["RESULTS", "REFERENCE_UNIT"]]
# tmp
plot_range(crrt_df, "CREATININE.*")  # the units are the same range

## Conversion/Exploration

In [ ]:
col = "METANEPHRINE"
display(crrt_df[crrt_df["COMPONENT_NAME"] == col])
display(cedars_df[cedars_df["COMPONENT_NAME"] == col])

In [ ]:
import matplotlib.pyplot as plt


def results_to_numeric(res: pd.Series) -> pd.Series:
    return pd.to_numeric(res.str.replace(">|<", ""), errors="coerce")


units_mapping = {
    # mg/dl -> g/dl
    "ALBUMIN": {
        # this looks weird with /1000 but right with /100
        "RESULTS": lambda df: results_to_numeric(df["RESULTS"]) / 100,
        "REFERENCE_UNIT": "g/dL",
    },
    # umol/l -> mcg/dl
    # molecular weight: https://unitslab.com/node/153
    "AMMONIA": {
        "RESULTS": lambda df: results_to_numeric(df["RESULTS"]) * 17.031 / 10,
        "REFERENCE_UNIT": "mcg/dL",
    },
    "ARSENIC, BLOOD": {},
}
for col, maping in units_mapping.items():
    print("=" * 100)
    # ucla_vals = crrt_df[crrt_df["DESCRIPTION"].str.contains(col)]
    # cedars_vals = cedars_df[cedars_df["COMPONENT_NAME"].str.contains(col)]
    ucla_vals = crrt_df[crrt_df["DESCRIPTION"] == col]
    cedars_vals = cedars_df[cedars_df["COMPONENT_NAME"] == col]
    # display(ucla_vals.head())
    # display(cedars_vals.head())
    results_to_numeric(ucla_vals["RESULTS"]).plot.area()
    results_to_numeric(cedars_vals["RESULTS"]).plot.area()
    plt.title(col)
    plt.show()
    plt.gcf().clear()

    results_to_numeric(ucla_vals["RESULTS"]).plot.area()
    cedars_vals.assign(**units_mapping[col])["RESULTS"].plot.area()
    plt.title(col)
    plt.show()
    plt.gcf().clear()

## Use Mapping File

In [3]:
from data.lab_proc_utils import align_units

aligned_df = align_units(crrt_df, args.ucla_crrt_data_dir)

mealy | WARNING - Redefining 'micro' (<class 'pint.delegates.txt_defparser.plain.PrefixDefinition'>)
mealy | WARNING - Redefining 'u' (<class 'pint.delegates.txt_defparser.plain.PrefixDefinition'>)


In [4]:
aligned_df["RESULTS"].equals(crrt_df["RESULTS"])

False

In [5]:
unique = aligned_df.groupby("COMPONENT_NAME")["REFERENCE_UNIT"].nunique()

In [ ]:
# aligned_df[uniqe[unique > 1]]
unique[unique > 1]

In [ ]:
aligned_df[aligned_df["COMPONENT_NAME"] == "ABSOLUTE BASOPHILS(QST)"][
    "REFERENCE_UNIT"
].unique()